In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

df = pd.read_csv("train.csv")
#X_test = pd.read_csv("test.csv")

df.prcp = df.prcp.fillna(0) # eeldan, et vihmasaju andme puudumine on sama mis 0
df.snow = df.snow.fillna(0) # eeldan, et lumesaju andme puudumine on sama mis 0

df = df.dropna()
#2021-09-01 00:00:00+03:00
df.time = pd.to_datetime(df['time'], format='%Y-%m-%dT%H:%M:%S%z')
df['Month'] = df['time'].apply(lambda time: time.month)
df['Day'] = df['time'].apply(lambda time: time.day)
df['Hour'] = df['time'].apply(lambda time: time.hour)
df["workday"] = df["time"].apply(lambda time: time.weekday() < 5)
df = df.drop("time", axis = 1)

df["is_dew"] = np.where(df["dwpt"] < df["temp"], True, False) # kas on udu või mitte

df_orig = df.copy(deep = True)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
for i in X_train:
    print(i)
dropped = []
results = []

temp
dwpt
rhum
prcp
snow
wdir
wspd
wpgt
pres
coco
el_price
Month
Day
Hour
workday
is_dew


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

lasso = Lasso()
lasso.fit(X_train, y_train)
predict = lasso.predict(X_test)
print("Lasso:", mean_absolute_error(y_test, predict))

ridge = Ridge().fit(X_train, y_train)
predict = ridge.predict(X_test)
print("Ridge:", mean_absolute_error(y_test, predict))

linear = LinearRegression().fit(X_train, y_train)
predict = linear.predict(X_test)
print("Linear:", mean_absolute_error(y_test, predict))

params = {'alpha': (np.linspace(0.001,300,1000))}

Lasso: 0.6362023420048444
Ridge: 0.6202793776967443
Linear: 0.6203209320999126


lasso = Lasso(max_iter = 7000) # best case is equal to linear
lasso_model = GridSearchCV(lasso, params, cv = 10)
lasso_model.fit(X_train, y_train)
print(lasso_model.best_params_)
print(lasso_model.best_score_)
print(mean_absolute_error(y_test, lasso_model.predict(X_test)))

ridge = Ridge() # best case is equal to linear
ridge_model = GridSearchCV(ridge, params, cv = 10)
ridge_model.fit(X_train, y_train)
print(ridge_model.best_params_)
print(ridge_model.best_score_)
print(mean_absolute_error(y_test, ridge_model.predict(X_test)))

In [3]:
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=70).fit(X_train, y_train)
print(mean_absolute_error(y_test, neigh.predict(X_test)))

0.6084557304517996


min = 99
k = 0
for i in np.arange(1,200, 2):
    neigh = KNeighborsRegressor(n_neighbors=i).fit(X_train, y_train)
    error = mean_absolute_error(y_test, neigh.predict(X_test))
    if (error < min):
        min = error
        k = i
print(k)
neigh = KNeighborsRegressor(n_neighbors=k).fit(X_train, y_train)
error = mean_absolute_error(y_test, neigh.predict(X_test))
print(error)

neigh = KNeighborsRegressor(n_neighbors=k, weights = "uniform").fit(X_train, y_train)
error = mean_absolute_error(y_test, neigh.predict(X_test))
print(error)
neigh = KNeighborsRegressor(n_neighbors=k, weights = "distance").fit(X_train, y_train)
error = mean_absolute_error(y_test, neigh.predict(X_test))
print(error)

In [4]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100, random_state=0)
rf.fit(X_train, y_train)
print(mean_absolute_error(y_test, rf.predict(X_test)))

0.5000372245384157


In [5]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt', "log2", 1]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [6]:
def calc(X_train, y_train, X_test, y_test):
    rf = RandomForestRegressor(n_estimators = 1200,random_state = 69, min_samples_split = 2, min_samples_leaf = 1, max_features = 'log2', max_depth = 90, bootstrap = False)
    rf.fit(X_train, y_train)
    print(mean_absolute_error(y_test, rf.predict(X_test)))
    results.append((mean_absolute_error(y_test, rf.predict(X_test))).round(4))
calc(X_train, y_train, X_test, y_test)
dropped.append("None")

0.4763521868175501


In [7]:
df = df_orig.copy(deep = True)
df = df.drop("wdir", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("Wind direction")

0.4739134589041097


In [8]:
df = df_orig.copy(deep = True)
df = df.drop("wspd", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("Windspeed")

0.4763658243994443


In [ ]:
df = df_orig.copy(deep = True)
df = df.drop("coco", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("Coco")

In [ ]:
df = df_orig.copy(deep = True)
df = df.drop("prcp", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("prcp")

In [ ]:
df = df_orig.copy(deep = True)
df = df.drop("dwpt", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("dwpt")

In [ ]:
df = df_orig.copy(deep = True)
df = df.drop("wpgt", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("Peak Windspeed")

In [ ]:
df = df_orig.copy(deep = True)
df = df.drop("dwpt", axis = 1)
df = df.drop("prcp", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("dwpt + prcp")

In [ ]:
df = df_orig.copy(deep = True)
df = df.drop("wspd", axis = 1)
df = df.drop("dwpt", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("wspd + dwpt")

In [ ]:
df = df_orig.copy(deep = True)
df = df.drop("wdir", axis = 1)
df = df.drop("prcp", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("wdir + prcp")

In [ ]:
df = df_orig.copy(deep = True)
df = df.drop("wdir", axis = 1)
df = df.drop("prcp", axis = 1)
df = df.drop("dwpt", axis = 1)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
calc(X_train, y_train, X_test, y_test)
dropped.append("wdir + prcp + dwpt")

In [ ]:
for i in range(sum( 1 for i in results)):
    print(results[i], dropped[i])

df = df_orig.copy(deep = True)

X = df.drop("consumption", axis = 1)
Y = df.consumption
X_train, X_test, y_train, y_test =train_test_split(X, Y, test_size=0.2, random_state = 69)
rf = RandomForestRegressor(random_state = 69)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 300, cv = 5, n_jobs = -1)
rf_random.fit(X_train, y_train)
print(rf_random.best_params_) # {'n_estimators': 1200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 90, 'bootstrap': False}
print(mean_absolute_error(y_test, rf_random.predict(X_test)))